In [22]:
import numpy as np

def matrix_generate(rows, columns, type_="full", eps=0):
    """
    matrix_generate(rows, columns, type_ = "full", eps=0)

    Создаёт случайную матрицу выбранного типа

    Если матрицу нужных размеров создать нельзя должен выдать
    строку f"Error with type {type_} and shape ({rows},{columns})".

    Parameters
    ----------
    rows : int
        Количество строк в создаваемой матрице.
    columns : int
        Количество столбцов в создаваемой матрице.
    type_ : str, optional
        Тип создаваемой матрицы: "full", "upper_triangular", "symmetric", "diagonal", 
        "lower_triangular", "singular", "stepped", "perturbation".
    eps: float, optional
        Дополнительное число, использующееся при генерации для некоторых типов матриц, таких как "perturbation".

    Returns
    -------
    out : ndarray or str
        Выдаёт матрицу нужного типа либо ошибку.

    Notes
    -----
    Поддерживаемые типы матриц:
        "full", "upper_triangular", "symmetric", "diagonal", "lower_triangular", 
        "singular", "stepped", "perturbation"
    """
    
    if type_ == "full":
        #cлучайная матрица
        return np.random.random(size=(rows, columns))

    elif type_ == "upper_triangular":
        #верхнетреугольная матрица
        A = np.random.random(size=(rows, columns))
        return np.triu(A)

    elif type_ == "lower_triangular":
        #нижнетреугольная матрица
        A = np.random.random(size=(rows, columns))
        return np.tril(A)

    elif type_ == "symmetric":
        #симметричная матрица
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        A = np.random.random(size=(rows, columns))
        return (A + A.T) / 2

    elif type_ == "diagonal":
        #диагональная матрица
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        return np.diag(np.random.random(size=rows))

    elif type_ == "singular":
        #вырожденная матрица (последняя колонка зависима от первой)
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        A = np.random.random(size=(rows, columns))
        A[:, -1] = A[:, 0]  #делаем зависимой последнюю колонку от первой
        return A

    elif type_ == "stepped":
        #матрица ступенчатого вида (нижняя треугольная с нулями выше главной диагонали)
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        return np.tril(np.random.random(size=(rows, columns)))

    elif type_ == "perturbation":
        #генерация матрицы с возмущениями, каждый элемент в диапазоне [-eps, eps]
        A = np.random.uniform(-eps, eps, size=(rows, columns))
        norm = np.linalg.norm(A, ord='fro')  #оценка нормы матрицы возмущений
        return A, norm

    else:
        return f"Error with type {type_} and shape ({rows},{columns})"


In [24]:
matrix_generate(1, 3)

array([[0.95369158, 0.54590922, 0.16998556]])

In [26]:
matrix_generate(4, 4, type_ = "upper_triangular")

array([[0.19843654, 0.28299344, 0.46608207, 0.36869335],
       [0.        , 0.94347268, 0.60228482, 0.94418427],
       [0.        , 0.        , 0.19383088, 0.03181252],
       [0.        , 0.        , 0.        , 0.39874645]])

In [28]:
matrix_generate(4, 3, type_ = "upper_triangular")

array([[0.32645621, 0.16405931, 0.34894916],
       [0.        , 0.84713651, 0.04022647],
       [0.        , 0.        , 0.03015023],
       [0.        , 0.        , 0.        ]])

In [30]:
matrix_generate(4, 4, type_ = "symmetric")

array([[0.31283467, 0.48519962, 0.69192793, 0.54134359],
       [0.48519962, 0.46553746, 0.40485372, 0.75443677],
       [0.69192793, 0.40485372, 0.72347887, 0.28742874],
       [0.54134359, 0.75443677, 0.28742874, 0.72202742]])

In [32]:
matrix_generate(4, 1, type_ = "symmetric")

'Error with type symmetric and shape (4,1)'

In [34]:
def degeneracy_probability(rows, columns, type_ = "full", eps = 0, n=1000):
    count_degenerate = 0
    for _ in range(n):
        if np.linalg.det(matrix_generate(rows, columns, type_, eps)) == 0:
            count_degenerate+=1
    return count_degenerate/n
        

In [38]:
print(degeneracy_probability(2, 2))

0.0


**Упражнение 2**

Реализовать вычисление трех основных норм векторов (L1, L2 и максимальную) и подчиненных им матричных норм. Реализовать вычисление числа обусловленности.

Примечание: для вычисления собственных значений можно использовать linalg.eigvals из модуля scipy.

In [52]:
def vector_norm(x, p=2):
    if p == 1:
        return np.sum(np.abs(x))
    elif p == 2:
        return np.sqrt(np.sum(np.pow(x, 2)))
    elif p == np.inf:
        return np.max(np.abs(x))


def matrix_norm(A, p=2):
    if p == 1:
        return np.max(np.sum(np.abs(A), axis=0))
    elif p == 2:
        return np.sqrt(np.max(np.linalg.eigvals(A.T @ A)))
    elif p == np.inf:
        return np.max(np.sum(np.abs(A), axis=1))


def matrix_conditioning_number(A, p=2):
    return matrix_norm(A, p) * matrix_norm(np.linalg.inv(A), p)

In [54]:
A = np.array([[1, 2],
              [3, 4]])

x = np.array([1, 2, 3])
print(f"L1 норма вектора [1, 2, 3]: {vector_norm(x, p=1)}")
print(f"L2 норма вектора [1, 2, 3]: {vector_norm(x)}")
print(f"Максимальная норма вектора [1, 2, 3]: {vector_norm(x, p=np.inf)}\n")

print(f"Матричная норма A (p=1): {matrix_norm(A, p=1)}")
print(f"Матричная норма A (p=2): {matrix_norm(A, p=2)}")
print(f"Матричная норма A (p=inf): {matrix_norm(A, p=np.inf)}\n")

print(f"Число обусловленности A (p=1): {matrix_conditioning_number(A, p=1)}")
print(f"Число обусловленности A (p=2): {matrix_conditioning_number(A)}")
print(f"Число обусловленности A (p=inf): {matrix_conditioning_number(A, p=np.inf)}")

L1 норма вектора [1, 2, 3]: 6


AttributeError: module 'numpy' has no attribute 'pow'